Setup

In [ ]:
import locale
import numpy as np
import pandas as pd
from tqdm import tqdm
from geopy.geocoders import Nominatim

In [ ]:
import plotly.express as px

In [ ]:
locale.setlocale(locale.LC_ALL, '')

In [ ]:
tqdm.pandas()

In [ ]:
f"Last executed: {pd.Timestamp.today(tz='Europe/Rome')}"

# Data

In [ ]:
store = "../data/houses.jl"

In [ ]:
%%time
df = pd.read_json(store, lines=True)

In [ ]:
df = df.set_index("_id")

In [ ]:
df["price"] = df.price.str.replace("€ ", "").str.replace(".", "").astype("float")

In [ ]:
df["area"] = df.area.str.replace(".", "").astype("float")

# Geocoding

## Retrieve location data

Run it just once to resolve addresses in geodata

Raw extraction of the address by removing words not address related

More than half location data were not retrieved by this approach

## Append location data

In [ ]:
location_data = pd.read_feather("../data/houses_location.feather").set_index("_id")

In [ ]:
location_data["lat"] = location_data.lat.astype("float")
location_data["lon"] = location_data.lon.astype("float")

In [ ]:
df = df.join(location_data)

Just data that can be displayed

In [ ]:
df = df[~df.lat.isna()]

In [ ]:
df["f_price"] = df.price.apply(lambda p: locale.currency(p, grouping=True))

In [ ]:
df["f_area"] = df.area.astype(str) + " m²"

In [ ]:
df["url"] = df.index.to_series().apply(lambda _id: f"<a target='_blank' href='https://www.immobiliare.it/annunci/{_id}/>link</a>'")

In [ ]:
df["rooms"] = df.rooms.str.replace("+","").astype("float")

In [ ]:
df.head(1)

# Exploration

In [ ]:
data = df[(140000 <= df.price) & (df.price <= 300000) & (df.rooms >= 2) & (df.rooms <= 3) & (df.area >= 50)]

In [ ]:
def rescale_area(area):
    _min, _max = area.min(), area.max()
    return (area - _min) / (_min + _max)

In [ ]:
fig = px.scatter_mapbox(data.reset_index(), lat="lat", lon="lon", hover_name="title", 
                        color="price", zoom=10.2, size=rescale_area(data.area.fillna(0)).round(1),
                        hover_data={"lat": False, "lon": False, 
                                    "_id": True, "f_price": True, 
                                    "f_area": True, "rooms": True, 
                                    "baths": True, "price": False, 
                                    "floor": True},
                        height=700, title="Milan housing")
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})